In [ ]:
import json
import sys
import random
import gc

sys.path.append("../")

from models.vector_model import vector_model
from tqdm import tqdm
from preprocessors.preprocessor import Preprocessor

import pandas as pd
import numpy as np

import IR_utils

random.seed(0)
np.random.seed(0)

%load_ext autoreload
%autoreload 2

In [ ]:
data_path = "../../data/dataset/corpus.jsonl"
docs = IR_utils.load_document_corpus(data_path=data_path, max_docs=500000)

print("Number of documents in corpus: {}".format(len(docs)))

preprocessor = Preprocessor()
tokenized_docs = preprocessor.preprocess(docs)

In [ ]:
vm = vector_model.create_model(documents=tokenized_docs, min_df=100)
vm

In [ ]:
vm.fit()
print(vm.describe())

In [ ]:
train_queries, all_queries = IR_utils.load_train_queries(
    "../../data/dataset/queries.jsonl", "../../data/task1_train.tsv"
)

test_queries, all_queries = IR_utils.load_test_queries(
    "../../data/dataset/queries.jsonl", "../../data/task1_test.tsv"
)

In [ ]:
TOP_K = 10

query_ids = list(test_queries.keys())

for query_id in tqdm(query_ids):
    d2v_query_answers = vm.find_similar(test_queries[query_id]["text"], TOP_K)

    retrieved_doc_ids = [id for id, score in d2v_query_answers]
    retrieved_doc_scores = [score for id, score in d2v_query_answers]

    test_queries[query_id]["retrieved_doc_ids"] = retrieved_doc_ids
    test_queries[query_id]["retrieved_doc_scores"] = retrieved_doc_scores